In [19]:
import tqdm
import torch
from torch.utils.data import DataLoader
import tensorflow as tf

from robomimic.utils.rlds_utils import droid_dataset_transform, robomimic_transform, TorchRLDSDataset

from octo.data.dataset import make_dataset_from_rlds, make_interleaved_dataset, make_single_dataset
from octo.data.utils.data_utils import combine_dataset_statistics
from octo.utils.spec import ModuleSpec

tf.config.set_visible_devices([], "GPU")
from octo.utils.spec import ModuleSpec
import deligrasp_dataset_transform

In [2]:
import importlib
import robomimic
import numpy as np
importlib.reload(robomimic.utils.rlds_utils)

<module 'robomimic.utils.rlds_utils' from 'c:\\workspace\\deligrasp_policy_learning\\robomimic\\utils\\rlds_utils.py'>

In [6]:
import tensorflow_datasets as tfds
builder = tfds.builder_from_directory(builder_dir=f"{DATA_PATH}/{DATASET_NAMES}/1.0.0")
ds = tfds.load(f"{DATASET_NAMES}", data_dir=f"{DATA_PATH}", split="train")
trajectory = next(iter(next(iter(ds))['steps']))
gripper_force = trajectory["action"][7]

In [22]:
importlib.reload(robomimic.utils.rlds_utils)
DATA_PATH = "C:/Users/willi/tensorflow_datasets"    # UPDATE WITH PATH TO RLDS DATASETS
DATASET_NAMES = "deligrasp_dataset"
EXP_LOG_PATH = "C:/workspace/deligrasp_policy_learning/logs" # UPDATE WITH PATH TO DESIRED LOGGING DIRECTORY
sample_weights = [1]

# import tensorflow_datasets as tfds
# builder = tfds.builder_from_directory(f"{DATA_PATH}/1.0.0")
# builder.info.features

BASE_DATASET_KWARGS = {
    "name": "deligrasp_dataset",
    "data_dir": DATA_PATH,
    "image_obs_keys": {"primary": "image", "secondary": "wrist_image"},
    "proprio_obs_key": "state",
    "language_key": "language_instruction",
    "action_proprio_normalization_type": "bounds",
    "action_normalization_mask": [True] * 9 + [False] * 2,      # don't normalize final (gripper) dimension
    "standardize_fn": ModuleSpec.create("robomimic.utils.rlds_utils:deligrasp_dataset_transform"),
    # "standardize_fn": droid_dataset_transform,
    # "train": True,
}

stats = make_dataset_from_rlds(**BASE_DATASET_KWARGS, train=True)
# combined_dataset_statistics = combine_dataset_statistics(
#     [make_dataset_from_rlds(**dataset_kwargs, train=True)[1] for dataset_kwargs in dataset_kwargs_list]
# )


In [24]:
stats

(<DLataset element_spec={'observation': {'image_primary': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'image_secondary': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'proprio': TensorSpec(shape=(None, 16), dtype=tf.float32, name=None), 'timestep': TensorSpec(shape=(None,), dtype=tf.int32, name=None)}, 'task': {'language_instruction': TensorSpec(shape=(None,), dtype=tf.string, name=None)}, 'action': TensorSpec(shape=(None, 11), dtype=tf.float32, name=None), 'dataset_name': TensorSpec(shape=(None,), dtype=tf.string, name=None)}>,
 {'action': {'mean': array([ 2.59525876e-07, -8.98353755e-06,  6.26606061e-06,  1.00000000e+00,
          -3.94223480e-06,  1.17622549e-05,  3.95866209e-06,  1.00000000e+00,
           1.83136490e-06, -1.90163068e-02,  1.45091570e-03]),
   'std': array([4.99562520e-05, 5.33768907e-05, 1.40456337e-04, 6.31753039e-07,
          1.29881650e-04, 1.99309536e-04, 1.29631822e-04, 5.01916190e-07,
          2.10918428e-04, 9.86317545e-02, 1.46769136e

In [25]:
dataset = make_single_dataset(
    BASE_DATASET_KWARGS,
    train=True,
    traj_transform_kwargs=dict(
        window_size=2,
        window_size=15,
        subsample_length=100,
        skip_unlabeled=True,            # skip all trajectories without language annotation
    ),
    frame_transform_kwargs=dict(
        image_augment_kwargs=dict(
        ),
        resize_size=dict(
            primary=[128, 128],
            secondary=[128, 128],
        ),
        num_parallel_calls=200,
    )
)

TypeError: apply_trajectory_transforms() got an unexpected keyword argument 'future_action_window_size'